# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121233e+02     1.484956e+00
 * time: 0.8002669811248779
     1     1.031812e+01     7.958426e-01
 * time: 2.2065999507904053
     2    -1.221804e+01     9.592555e-01
 * time: 2.9066319465637207
     3    -3.460875e+01     7.019595e-01
 * time: 3.9215869903564453
     4    -4.797684e+01     5.711330e-01
 * time: 4.9702019691467285
     5    -5.701119e+01     2.191472e-01
 * time: 5.954797029495239
     6    -5.972144e+01     1.971720e-01
 * time: 6.615785837173462
     7    -6.087288e+01     6.968474e-02
 * time: 7.319634914398193
     8    -6.139612e+01     3.712148e-02
 * time: 8.004385948181152
     9    -6.171951e+01     3.040361e-02
 * time: 8.670706033706665
    10    -6.190830e+01     2.398944e-02
 * time: 9.356279850006104
    11    -6.202859e+01     2.074101e-02
 * time: 10.02497386932373
    12    -6.209744e+01     1.677366e-02
 * time: 10.692651987075806
    13    -6.214311e+01     1.233604e-02
 * time: 11.392343997955322


In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671062
    AtomicLocal         -18.8557697
    AtomicNonlocal      14.8522671
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666461558